In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-29"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14142,2024-03-29,Eua Nba,20:00,Indiana Pacers,Los Angeles Lakers,1.74,1.93,239.5,1.83,1.97,-2.5,1.96,2.26,bsuzCfO8,0.574713,0.518135,0.546448,0.507614,0.092847,0.2,0.4,NaN,NaN,236.658,63.143368,0.266813,234.272,39.838108,0.170051,187.228,222.228,214.23,221.68,0.0,0.0,0.0,0.0,0.073215,0.052103,0.100537,-0.55,-0.110,-6.727273,0.565491,0.6,0.034509,0.18,0.036,25.833333,0.474534,0.5,0.025466
14143,2024-03-29,Eua Nba,20:00,Orlando Magic,Los Angeles Clippers,1.98,1.88,208.5,1.81,1.97,-1.5,2.05,1.98,SGqvBEwF,0.505051,0.531915,0.552486,0.507614,0.036965,0.6,0.2,NaN,NaN,170.976,34.962239,0.204486,203.408,36.678217,0.180318,182.296,207.138,142.29,249.10,0.0,0.0,0.0,0.0,0.036638,0.059861,0.024565,-1.43,-0.286,-3.426573,0.598895,0.6,0.001105,-0.06,-0.012,-73.333333,0.607054,0.7,0.092946
14144,2024-03-29,Eua Nba,20:00,Washington Wizards,Detroit Pistons,1.66,2.04,228.5,1.89,1.97,-3.5,2.05,2.43,67rrAYhL,0.602410,0.490196,0.529101,0.507614,0.092606,0.4,0.4,NaN,NaN,418.574,323.600334,0.773102,456.920,214.563901,0.469587,367.522,643.046,242.76,608.00,0.0,1.0,0.0,0.0,0.145244,0.029310,0.119956,-3.29,-0.658,-1.003040,0.290600,0.1,-0.190600,5.55,1.110,0.936937,0.218128,0.3,0.081872
14145,2024-03-29,Eua Nba,21:00,Oklahoma City Thunder,Phoenix Suns,1.71,1.98,229.5,1.84,1.96,-2.5,1.88,2.13,rmxM2joe,0.584795,0.505051,0.543478,0.510204,0.089846,0.6,0.4,NaN,NaN,185.404,30.293862,0.163394,219.400,49.178441,0.224150,172.688,234.786,132.68,279.93,0.0,0.0,0.0,0.0,0.103479,0.044659,0.088168,1.73,0.346,2.052023,0.773729,0.9,0.126271,-2.93,-0.586,-1.672355,0.565193,0.4,-0.165193
14146,2024-03-29,Eua Nba,22:30,Utah Jazz,Houston Rockets,2.16,1.59,225.5,1.86,1.94,1.5,2.03,1.61,hIJCPDh8,0.462963,0.628931,0.537634,0.515464,0.091894,0.6,0.4,NaN,NaN,217.516,30.762554,0.141427,215.020,44.549657,0.207188,227.044,204.852,220.72,163.77,0.0,0.0,0.0,0.0,0.214960,0.029773,0.163178,-1.24,-0.248,-4.677419,0.565537,0.5,-0.065537,-1.97,-0.394,-1.497462,0.490804,0.3,-0.190804
14147,2024-03-29,Eua Nba,23:00,Sacramento Kings,Dallas Mavericks,1.74,1.93,228.5,1.85,1.95,-2.5,2.04,2.13,6LNGOX7E,0.574713,0.518135,0.540541,0.512821,0.092847,0.2,0.4,NaN,NaN,209.500,43.494586,0.207611,223.926,43.669245,0.195016,191.148,234.820,175.68,269.28,0.0,0.0,0.0,0.0,0.073215,0.037216,0.030523,0.00,0.000,inf,0.684249,0.6,-0.084249,0.37,0.074,12.567568,0.546491,0.6,0.053509
14148,2024-03-29,Europa Euroliga,16:30,Virtus Bologna,Panathinaikos,2.20,1.78,160.5,1.85,2.09,1.5,2.01,1.64,djWqrAj6,0.454545,0.561798,0.540541,0.478469,0.016343,0.4,0.4,NaN,NaN,167.804,20.928294,0.124719,174.294,86.094623,0.493962,144.584,175.564,185.74,138.84,0.0,0.0,0.0,0.0,0.149239,0.086145,0.143359,-0.44,-0.088,-13.636364,0.571863,0.8,0.228137,-0.07,-0.014,-55.714286,0.511743,0.6,0.088257
14149,2024-03-29,Europa Eurocopa,16:30,London Lions,Paris,2.26,1.59,176.5,1.86,1.84,2.5,1.96,1.51,vJexVivg,0.442478,0.628931,0.537634,0.543478,0.071409,0.2,0.8,NaN,NaN,109.858,17.067990,0.155364,132.848,25.112786,0.189034,117.450,150.540,93.24,160.55,0.0,0.0,0.0,0.0,0.246110,0.007644,0.183399,-1.24,-0.248,-5.080645,0.801970,0.7,-0.101970,2.49,0.498,1.184739,0.000000,0.0,0.000000
14150,2024-03-29,Bulgária Nbl,14:00,Minyor,Levski,1.77,1.95,152.5,1.81,1.89,-2.5,1.98,1.92,UNdFLMOE,0.564972,0.512821,0.552486,0.529101,0.077792,0.4,0.4,NaN,NaN,202.128,57.902521,0.286465,296.624,105.003520,0.353995,376.440,268.436,180.60,217.60,0.0,0.0,0.0,0.0,0.068430,0.030578,0.021757,7.07,1.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Indiana Pacers,Los Angeles Lakers,239.5,1.83,1.0000,Over
1,20:00,Eua Nba,Orlando Magic,Los Angeles Clippers,208.5,1.81,0.9999,Over
2,20:00,Eua Nba,Washington Wizards,Detroit Pistons,228.5,1.89,1.0000,Over
3,21:00,Eua Nba,Oklahoma City Thunder,Phoenix Suns,229.5,1.84,0.9999,Over
4,23:00,Eua Nba,Sacramento Kings,Dallas Mavericks,228.5,1.85,1.0000,Over
5,07:00,Coreia Do Sul Kbl,KCC Egis,Mobis Phoebus,177.5,1.86,1.0000,Over
6,16:00,Espanha Leb Ouro,Estela,Oviedo,157.5,1.85,1.0000,Over
7,20:39,Eua Ncaa,Purdue,Gonzaga,154.5,1.91,1.0000,Over
8,23:09,Eua Ncaa,Tennessee,Creighton,143.5,1.80,1.0000,Over
9,16:00,Europa Liga Bnxt,Den Bosch,Limburg,148.5,1.84,1.0000,Over
